In [13]:
# Import libraries
from openmm.app import *
from openmm import *
from openmm.unit import *
import openmmml
import sys

import importlib
importlib.reload(openmmml)

# Setup
pdb = PDBFile("../pdbs/aaa.pdb")
potential = openmmml.MLPotential('ani2x')
system = potential.createSystem(pdb.topology)

/home/matthewb/miniconda/envs/ani/lib/python3.9/site-packages/torchani/resources/


In [12]:
# Import libraries
from openmm.app import *
from openmm import *
from openmm.unit import *
import openmmml
import sys

import importlib
importlib.reload(openmmml)

pdb = PDBFile("../pdbs_equilibrated/aaa_amber.pdb")
# pdb.topology.setPeriodicBoxVectors(None)

amber_system = forcefield = ForceField(
    'amber14-all.xml',
    'amber14/tip3p.xml'
).createSystem(
    pdb.topology,
    nonbondedMethod=PME,
    nonbondedCutoff=1*nanometer,
    constraints=AllBonds,
    hydrogenMass=4*amu
)

ani_atoms = [atom.index for atom in pdb.topology.atoms() if atom.residue.name != "HOH"]

# Setup
potential = openmmml.MLPotential('ani2x')
system = potential.createMixedSystem(
    pdb.topology,
    amber_system,
    ani_atoms
)


/home/matthewb/miniconda/envs/ani/lib/python3.9/site-packages/torchani/resources/


# Works fine on CPU (slow)

In [15]:
integrator = LangevinIntegrator(
    300 * kelvin, 
    1 / picosecond, 
    1.0 * femtosecond,
)
simulation = Simulation(
    pdb.topology,
    system,
    integrator,
    # Platform.getPlatformByName("CPU"),
)
STEPS = 1000
simulation.reporters.append(
    StateDataReporter(
        sys.stdout, 
        30, 
        totalSteps=STEPS,
        speed=True,
        progress=True,
        remainingTime=True,
    )
)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.step(STEPS)
print("done")

#"Progress (%)","Speed (ns/day)","Time Remaining"
3.0%,0,--
6.0%,29.7,0:02
9.0%,30.3,0:02
12.0%,30.4,0:02
15.0%,30.5,0:02
18.0%,30.6,0:02
21.0%,30.5,0:02
24.0%,30.5,0:02
27.0%,30.5,0:02
30.0%,30.4,0:01
33.0%,30.3,0:01
36.0%,30.3,0:01
39.0%,30.3,0:01
42.0%,30.3,0:01
45.0%,30.3,0:01
48.0%,30.3,0:01
51.0%,30.3,0:01
54.0%,30.3,0:01
57.0%,30.3,0:01
60.0%,30.3,0:01
63.0%,30.3,0:01
66.0%,30.4,0:00
69.0%,30.4,0:00
72.0%,30.4,0:00
75.0%,30.4,0:00
78.0%,30.4,0:00
81.0%,30.4,0:00
84.0%,30.4,0:00
87.0%,30.4,0:00
90.0%,30.4,0:00
93.0%,30.4,0:00
96.0%,30.3,0:00
99.0%,30.3,0:00
done


# Breaks with CUDA

In [10]:
integrator = LangevinIntegrator(
    300 * kelvin, 
    1 / picosecond, 
    1.0 * femtosecond,
)
simulation = Simulation(
    pdb.topology,
    system,
    integrator,
    Platform.getPlatformByName("CUDA"),
)
STEPS = 1000
simulation.reporters.append(
    StateDataReporter(
        sys.stdout, 
        30, 
        totalSteps=STEPS,
        speed=True,
        progress=True,
        remainingTime=True,
    )
)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.step(STEPS)
print("done")

#"Progress (%)","Speed (ns/day)","Time Remaining"
3.0%,0,--
6.0%,27.8,0:02
9.0%,27.9,0:02
12.0%,27.8,0:02
15.0%,27.9,0:02
18.0%,27.9,0:02
21.0%,27.9,0:02
24.0%,27.8,0:02
27.0%,27.8,0:02
30.0%,27.8,0:02
33.0%,27.8,0:02
36.0%,27.8,0:01
39.0%,27.9,0:01
42.0%,27.9,0:01
45.0%,27.9,0:01
48.0%,27.9,0:01
51.0%,27.9,0:01
54.0%,27.9,0:01
57.0%,28,0:01
60.0%,28,0:01
63.0%,28,0:01
66.0%,28,0:01
69.0%,28,0:00
72.0%,28,0:00
75.0%,28,0:00
78.0%,28,0:00
81.0%,28.1,0:00
84.0%,28.1,0:00
87.0%,28.1,0:00
90.0%,28.1,0:00
93.0%,28.1,0:00
96.0%,28.1,0:00
99.0%,28.1,0:00
done


# Troubleshooting

In [8]:
!conda list | grep -E "torch|openmm"

/bin/bash: conda: command not found


In [4]:
import torch

print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

False


/home/matthewb/miniconda/envs/ani/lib/python3.9/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1640869844479/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.